In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as ipw
from jinja2 import Environment
from importlib_resources import files

from aiidalab_widgets_base import WizardAppWidget
from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

from aiidalab_sssp import static
from aiidalab_sssp.steps import SubmitVerificationStep
from aiidalab_sssp.steps import ViewSsspAppWorkChainStatusAndResultsStep
from aiidalab_sssp.steps import PseudoInputStep

pseudo_input_step = PseudoInputStep()
submit_verification_work_chain_step = SubmitVerificationStep()
view_sssp_app_work_chain_status_and_results_step = ViewSsspAppWorkChainStatusAndResultsStep()

# Link the application steps
ipw.dlink((pseudo_input_step, 'confirmed_pseudo'), (submit_verification_work_chain_step, 'input_pseudo'))
ipw.dlink((submit_verification_work_chain_step, 'process'), (view_sssp_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Input pesudo', pseudo_input_step),
        ('Submit verification work chain', submit_verification_work_chain_step),
        ('Status & Results', view_sssp_app_work_chain_status_and_results_step),
    ])

# Reset all subsequent steps in case that a new pseudo is selected
def _observe_pseudo_selection(change):
    with pseudo_input_step.hold_sync():
        if pseudo_input_step.confirmed_pseudo is not None and \
                pesudo_input_step.confirmed_pseudo != change['new']:
            app.reset()
            
pseudo_input_step.observe(_observe_pseudo_selection, 'pseudo')

app_with_work_chain_selector = ipw.VBox(children=[app])

env = Environment()

template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()
welcome_message = ipw.HTML(env.from_string(template).render(style=style))

app_with_work_chain_selector = ipw.VBox(children=[app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/aiidalab/aiidalab-sssp/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(welcome_message, app_with_work_chain_selector)
    
display(output)

Output()